In [24]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

In [19]:
from sklearn.model_selection import train_test_split

## Load datasets

In [16]:
dataset_path = '../../dataset/additional_datasets/'
dataset_name = 'Breast_GSE45827'
dataset_norm_suffix = '_norm'
dataset_pca_suffix = '_pca'

In [17]:
df_norm = pd.read_csv(dataset_path + dataset_name + dataset_norm_suffix + '.csv', index_col=0)
df_pca = pd.read_csv(dataset_path + dataset_name + dataset_pca_suffix + '.csv', index_col=0)

## Train

In [32]:
def train(df):
    # Train xgboost on the dataset
    X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.2, random_state=42)
    
    # create model instance
    model = XGBClassifier(objective='multi:softmax', num_class=6, random_state=42)
    # fit model
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    return preds, y_test

In [33]:
norm_pred, norm_real = train(df_norm)
pca_pred, norm_real = train(df_pca)

c:\Users\Branko\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\Branko\miniconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[21:28:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:29:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


## Results

In [34]:
def get_accuracy(pred, real):
    return np.sum(pred == real) / len(pred)

print('Accuracy of norm dataset: ', get_accuracy(norm_pred, norm_real))
print('Accuracy of pca dataset: ', get_accuracy(pca_pred, norm_real))

Accuracy of norm dataset:  0.0
Accuracy of pca dataset:  0.16129032258064516
